전체적인 구조 설명
전고점이 돌파했을 경우 매수하여서 수익을 보는 구조입니다.
전고점의 기준은 이전날 최고점을 기준으로 합니다.
비트코인이 꾸준히 우상향하고, 이전 고점을 넘어섰을 경우 상승추세라는 가정하에 진행하게 되었습니다.
매수수량을 넘어서는 거래가 나와서 전체 거래량의 0.01%를 넘지 않는 기준에서 거래한다는 가정하에 진행되었습니다.

Highbreack_ideal.py로 만약에 전날 고점에서 사서 당일 고점에서 팔았을 경우의 수익률을 계산했습니다.
2020년 1월부터 2024년 11월 26일까지의 계산 결과로 초기금액 100달러 수익 금액 총합: 6,077,866.07달러, 6,077,866.07%가 나왔습니다.
가장 이상적인 형태로 고점을 예측한다는 가정하에 이러한 수익률을 얻을 수 있다는 점만 알았으면 합니다.

In [ ]:
import pandas as pd
import backtrader as bt

# CSV 파일 경로
file_path = "./Recent_BTC_1day.csv"

# CSV 파일을 pandas 데이터프레임으로 불러오기
btc_data = pd.read_csv(file_path)

# 열 이름을 backtrader에서 사용하기 쉽게 변경
btc_data = btc_data.rename(columns={
    'Open_time': 'datetime',
    '시가': 'open',
    '고가': 'high', 
    '저가': 'low',
    '종가': 'close',
    '거래량': 'volume'
})

# backtrader에 필요 없는 열 제거
btc_data = btc_data[['datetime', 'open', 'high', 'low', 'close', 'volume']]

# datetime 컬럼을 datetime 타입으로 변환하고 인덱스로 설정
btc_data['datetime'] = pd.to_datetime(btc_data['datetime'])
btc_data.set_index('datetime', inplace=True)

# backtrader 데이터피드로 변환
class CustomData(bt.feeds.PandasData):
    params = (
        ('datetime', None),
        ('open', 'open'),
        ('high', 'high'),
        ('low', 'low'),
        ('close', 'close'),
        ('volume', 'volume'),
        ('openinterest', -1),
    )

data_feed = CustomData(dataname=btc_data)

# Cerebro 초기 설정 및 데이터 추가
cerebro = bt.Cerebro(optreturn=False, stdstats=False)  # 최적화를 위한 옵션 설정
cerebro.broker.setcommission(commission=0.05 / 100, leverage=1)  # upbit 수수료인 0.05% 수수료 설정
cerebro.broker.setcash(100)
cerebro.broker.set_coc(True)  # 종가 진입 설정

# 데이터 피드 추가
cerebro.adddata(data_feed, name="BTC")

# 전략을 쉽게 추가할 수 있도록 분리된 전략 뼈대 클래스
class HighBreakoutStrategy(bt.Strategy):
    # 전략에서 사용할 매개변수 정의
    params = dict(
        log=True,  # 로그 출력 여부
    )

    def log(self, txt, dt=None):
        '''로깅 기능'''
        dt = dt or self.datas[0].datetime.date(0)
        print(f'{dt.isoformat()} {txt}')

    def next(self):
        # 오늘과 어제 데이터 확인
        if len(self.data) < 2:
            return  # 데이터를 충분히 확보하지 못한 경우 종료
        
        # 어제와 오늘의 고가
        yesterday_high = self.data.high[-1]  # 전날 고가
        today_high = self.data.high[0]  # 오늘 고가
        
        # 조건: 오늘 고가가 어제 고가보다 높은 경우
        if today_high > yesterday_high:
            # 매수: 어제 고가로 매수
            buy_price = yesterday_high
            order_size = self.broker.getcash() / buy_price  # 포트폴리오 전체를 사용할 경우
            
            # 매수 주문
            self.buy(size=order_size, price=buy_price)
            
            # 매도: 오늘 고가로 매도
            sell_price = today_high
            self.sell(size=order_size, price=sell_price)
            
            # 로그 출력
            if self.p.log:
                self.log(f'[매수] 어제 고가: {yesterday_high:.2f}, [매도] 오늘 고가: {today_high:.2f}')

# 전략 추가 (여기에서 사용자가 원하는 전략을 선택할 수 있음)
cerebro.addstrategy(HighBreakoutStrategy)
cerebro.addanalyzer(bt.analyzers.PyFolio, _name='pyfolio')  # 결과 분석 추가

# 백테스트 실행
print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())

results = cerebro.run()
pyfoliozer = results[0].analyzers.getbyname('pyfolio')

# 결과 처리
returns, positions, transactions, gross_lev = pyfoliozer.get_pf_items()
returns.index = returns.index.tz_convert(None)

print(f'\nFinal Portfolio Value {cerebro.broker.getvalue()}')

# 수익률 시각화 및 보고서 생성
import quantstats as qs
qs.plots.snapshot(returns)

print("\nGenerating report...")
import os
if not os.path.exists("./results"):
    os.makedirs("./results")
qs.reports.html(returns, output=f'./results/Highbreack_ideal.html',
                download_filename=f'./results/Highbreack_ideal.html',
                title="Highbreack_ideal")
print("Complete")

Highbreack_end.py 로 단순하게 전날 고점이 돌파하였을 경우 매수하여서 종가에 매도하는 전략입니다
2020년 1월부터 2024년 11월 26일까지의 계산 결과로 초기금액 100달러 수익 금액 총합: 917,080.11 수익률: 917,080.11%가 나왔습니다.

In [ ]:
import pandas as pd
import backtrader as bt

# CSV 파일 경로
file_path = "./Recent_BTC_1day_real.csv"

# CSV 파일을 pandas 데이터프레임으로 불러오기
btc_data = pd.read_csv(file_path)

# 열 이름을 backtrader에서 사용하기 쉽게 변경
btc_data = btc_data.rename(columns={
    'Open_time': 'datetime',
    '시가': 'open',
    '고가': 'high', 
    '저가': 'low',
    '종가': 'close',
    '거래량': 'volume'
})

# backtrader에 필요 없는 열 제거
btc_data = btc_data[['datetime', 'open', 'high', 'low', 'close', 'volume']]

# datetime 컬럼을 datetime 타입으로 변환하고 인덱스로 설정
btc_data['datetime'] = pd.to_datetime(btc_data['datetime'])
btc_data.set_index('datetime', inplace=True)

# backtrader 데이터피드로 변환
class CustomData(bt.feeds.PandasData):
    params = (
        ('datetime', None),
        ('open', 'open'),
        ('high', 'high'),
        ('low', 'low'),
        ('close', 'close'),
        ('volume', 'volume'),
        ('openinterest', -1),
    )

data_feed = CustomData(dataname=btc_data)

# Cerebro 초기 설정 및 데이터 추가
cerebro = bt.Cerebro(optreturn=False, stdstats=False)  # 최적화를 위한 옵션 설정
cerebro.broker.setcommission(commission=0.05 / 100, leverage=1)  # upbit 수수료인 0.05% 수수료 설정
cerebro.broker.setcash(100)
cerebro.broker.set_coc(True)  # 종가 진입 설정

# 데이터 피드 추가
cerebro.adddata(data_feed, name="BTC")

class HighBreakoutStrategy(bt.Strategy):
    # 전략에서 사용할 매개변수 정의
    params = dict(
        log=True,  # 로그 출력 여부
    )

    def log(self, txt, dt=None):
        '''로깅 기능'''
        dt = dt or self.datas[0].datetime.date(0)
        print(f'{dt.isoformat()} {txt}')

    def next(self):
        # 오늘과 어제 데이터 확인
        if len(self.data) < 2:
            return  # 데이터를 충분히 확보하지 못한 경우 종료
        
        # 어제와 오늘의 고가 및 오늘의 거래량
        yesterday_high = self.data.high[-1]  # 전날 고가
        today_high = self.data.high[0]  # 오늘 고가
        today_close = self.data.close[0]  # 오늘 종가
        today_volume = self.data.volume[0]  # 오늘 거래량
        
        # 조건: 오늘 고가가 어제 고가보다 높은 경우
        if today_high > yesterday_high:
            # 매수: 어제 고가로 매수
            buy_price = yesterday_high
            order_size = self.broker.getcash() / buy_price  # 포트폴리오 전체를 사용할 경우
            
            # 거래량 제한 (거래량의 0.01% 이하)
            max_order_size = today_volume * 0.0001
            order_size = min(order_size, max_order_size)
            
            # 매수 주문
            self.buy(size=order_size, price=buy_price)
            
            # 매도: 오늘 종가로 매도
            sell_price = today_close
            self.sell(size=order_size, price=sell_price)
            
            # 로그 출력
            if self.p.log:
                profit = (sell_price - buy_price) * order_size  # 수익 금액 계산
                self.log(
                    f'[매수] 어제 고가: {yesterday_high:.2f}, [매도] 오늘 종가: {today_close:.2f}, '
                    f'거래량 제한: {max_order_size:.4f}, 매수 수량: {order_size:.4f}, '
                    f'수익 금액: {profit:.2f}'
                )


# 전략 추가 (여기에서 사용자가 원하는 전략을 선택할 수 있음)
cerebro.addstrategy(HighBreakoutStrategy)
cerebro.addanalyzer(bt.analyzers.PyFolio, _name='pyfolio')  # 결과 분석 추가

# 백테스트 실행
print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())

results = cerebro.run()
pyfoliozer = results[0].analyzers.getbyname('pyfolio')

# 결과 처리
returns, positions, transactions, gross_lev = pyfoliozer.get_pf_items()
returns.index = returns.index.tz_convert(None)

# 최종 포트폴리오와 현금 확인
final_portfolio_value = cerebro.broker.getvalue()  # 총 포트폴리오 가치
final_cash = cerebro.broker.getcash()  # 남아 있는 현금

print(f'\nFinal Portfolio Value: {final_portfolio_value:.2f}')
print(f'Final Cash in Hand: {final_cash:.2f}')

import quantstats as qs
qs.plots.snapshot(returns)

print("\nGenerating report...")

import os
if not os.path.exists("./results"):
    os.makedirs("./results")
qs.reports.html(returns, output=f'./results/Highbreack_end.html',
                download_filename=f'./results/Highbreack_end.html',
                title="Highbreack_ideal")
print("Complete")

Highbreack_end_per_mixed.py 로 종가에 매도하는 방식은 비슷하지만 폭락을 대비하여서 하락폭을 지정한 다음 그 이상을 넘어서는 경우 매도하도록 하였습니다.
기존 결과에서는 2021.5.12, 2022.9.13에 많은 손실을 보았는데 다음을 보안해 줍니다

오늘 고점이 이전 고점을 넘어섰을 경우 오늘 고점은 이전 고점보다 평균적으로 2.478% 높습니다. 표준편차는 2.835로 지정됩니다
그러한 결과로 95%범위를 벗어난 경우 손절했다는 방식으로 매도가격을 조정했습니다.
2020년 1월부터 2024년 11월 26일까지의 계산 결과로 초기금액 100달러 수익 금액 총합: 1,491,774.12 수익률: 1,491,774.12%가 나왔습니다.

하지만 오늘 고점이 당연히 어제의 고점보다 높아서 양수인데 표준편차에 -1을 곱해서 이전값들을 해도 -0.357 ~ 2.48 의 범위가 34.1%인 것을 보고 상승폭이 너무 큰 것들이 있어서 너무 부정확하다는 것을 깨달았습니다
확인해 본 결과 최대값이 22.289인 경우도 있어서 이상치(표준편차보다 3배 더 큰 경우)를 제외하고 다시 평균과 표준편차를 계산해 본 결과
이상치를 제거한 후 평균 1.973%, 표준편차: 1.752를 확인하였습니다
그래서 95%의 범위인 1.973 - 1.752*1.96 보다 큰 경우가 전체의 95%라는 계산 아래에 그 아래는 손절했다는 기준으로 진행하였습니다.
2020년 1월부터 2024년 11월 26일까지의 계산 결과로 초기금액 100달러 수익 금액 총합: 2,114,625.90 수익률: 2,114,625.90%가 나왔습니다.

In [ ]:
import pandas as pd
import backtrader as bt

# CSV 파일 경로
file_path = "./Recent_BTC_1day_real.csv"

# CSV 파일을 pandas 데이터프레임으로 불러오기
btc_data = pd.read_csv(file_path)

# 열 이름을 backtrader에서 사용하기 쉽게 변경
btc_data = btc_data.rename(columns={
    'Open_time': 'datetime',
    '시가': 'open',
    '고가': 'high', 
    '저가': 'low',
    '종가': 'close',
    '거래량': 'volume'
})

# backtrader에 필요 없는 열 제거
btc_data = btc_data[['datetime', 'open', 'high', 'low', 'close', 'volume']]

# datetime 컬럼을 datetime 타입으로 변환하고 인덱스로 설정
btc_data['datetime'] = pd.to_datetime(btc_data['datetime'])
btc_data.set_index('datetime', inplace=True)

# backtrader 데이터피드로 변환
class CustomData(bt.feeds.PandasData):
    params = (
        ('datetime', None),
        ('open', 'open'),
        ('high', 'high'),
        ('low', 'low'),
        ('close', 'close'),
        ('volume', 'volume'),
        ('openinterest', -1),
    )

data_feed = CustomData(dataname=btc_data)

# Cerebro 초기 설정 및 데이터 추가
cerebro = bt.Cerebro(optreturn=False, stdstats=False)  # 최적화를 위한 옵션 설정
cerebro.broker.setcommission(commission=0.05 / 100, leverage=1)  # upbit 수수료인 0.05% 수수료 설정
cerebro.broker.setcash(100)
cerebro.broker.set_coc(True)  # 종가 진입 설정

# 데이터 피드 추가
cerebro.adddata(data_feed, name="BTC")

class HighBreakoutStrategy(bt.Strategy):
    # 전략에서 사용할 매개변수 정의
    params = dict(
        log=True,  # 로그 출력 여부
        stop_loss_pct=-1.46092  # -1.46092% 이하로 떨어지지 않도록 하는 비율 설정
    )

    def log(self, txt, dt=None):
        '''로깅 기능'''
        dt = dt or self.datas[0].datetime.date(0)
        print(f'{dt.isoformat()} {txt}')

    def next(self):
        # 오늘과 어제 데이터 확인
        if len(self.data) < 2:
            return  # 데이터를 충분히 확보하지 못한 경우 종료
        
        # 어제와 오늘의 고가
        yesterday_high = self.data.high[-1]  # 전날 고가
        today_high = self.data.high[0]  # 오늘 고가
        today_close = self.data.close[0]  # 오늘 종가
        today_volume = self.data.volume[0]  # 오늘 거래량
        
        
        # 조건: 오늘 고가가 어제 고가보다 높은 경우
        if today_high > yesterday_high:
            # 매수: 어제 고가로 매수
            buy_price = yesterday_high
            order_size = self.broker.getcash() / buy_price  # 포트폴리오 전체를 사용할 경우
            
            # 거래량 제한 (거래량의 0.01% 이하)
            max_order_size = today_volume * 0.0001
            order_size = min(order_size, max_order_size)

            # 매수 주문
            self.buy(size=order_size, price=buy_price)
            
            # 매도: 오늘 종가로 매도
            sell_price = today_close
            # 수익률이 -1.46092% 이하로 떨어지지 않도록 수정
            stop_loss_price = buy_price * (1 + self.p.stop_loss_pct / 100)  # -1.46092% 제한
            if sell_price < stop_loss_price:
                sell_price = stop_loss_price  # 강제로 -1.46092%로 설정

            # 매도 주문
            self.sell(size=order_size, price=sell_price)
            
            # 로그 출력
            if self.p.log:
                profit = (sell_price - buy_price) * order_size  # 수익 금액 계산
                self.log(
                    f'[매수] 어제 고가: {yesterday_high:.2f}, [매도] 오늘 종가: {today_close:.2f}, '
                    f'거래량 제한: {max_order_size:.4f}, 매수 수량: {order_size:.4f}, '
                    f'수익 금액: {profit:.2f}'
                )



# 전략 추가 (여기에서 사용자가 원하는 전략을 선택할 수 있음)
cerebro.addstrategy(HighBreakoutStrategy)
cerebro.addanalyzer(bt.analyzers.PyFolio, _name='pyfolio')  # 결과 분석 추가

# 백테스트 실행
print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())

results = cerebro.run()
pyfoliozer = results[0].analyzers.getbyname('pyfolio')

# 결과 처리
returns, positions, transactions, gross_lev = pyfoliozer.get_pf_items()
returns.index = returns.index.tz_convert(None)

# 최종 포트폴리오와 현금 확인
final_portfolio_value = cerebro.broker.getvalue()  # 총 포트폴리오 가치
final_cash = cerebro.broker.getcash()  # 남아 있는 현금

print(f'\nFinal Portfolio Value: {final_portfolio_value:.2f}')
print(f'Final Cash in Hand: {final_cash:.2f}')

# 수익률 시각화 및 보고서 생성
import quantstats as qs
qs.plots.snapshot(returns)

print("\nGenerating report...")
import os
if not os.path.exists("./results"):
    os.makedirs("./results")
qs.reports.html(returns, output=f'./results/Highbreack_ideal.html',
                download_filename=f'./results/Highbreack_ideal.html',
                title="Highbreack_ideal")
print("Complete")

Highbreack_per_real.py는 위의 경우 코드의 경우 이상적인 상황에서의 손절했을 경우였습니다.
그 전에 자료를 조사한 후 도달값을 정하기로 하였습니다
결론적으로는 코드를 완성하진 못하고 조사와 이론만 세우고 끝났습니다....
조사는 대충 읽으셔도 되고, 결과만 확인해주시면 감사하겠습니다

2018~ 전체 고가들의 변동
고가 변동률 평균: 0.279%
고가 변동률 표준편차: 2.339%
변동률이 1%보다 클 확률: 25.99%

2018~ 고가 갱신후
고가 변동률 평균: 0.611%
고가 변동률 표준편차: 3.327
변동률이 0.05%보다 클 확률: 64.12%
변동률이 0.5%보다 클 확률: 49.41%
변동률이 1%보다 클 확률: 41.18%
애매하긴 하지만 수익을 얻을 수 있는 확률이 평균보다 약간 못미치고, 기다려야 하는 시간과 함께 생각하면 수익이 나오지 않을 것 같아 구상을 포기하였습니다.

stop_loss를 설정하면
변동률을 활용한 계산값 : 0.611% - 1.96*(2.339 = 2.34) = -3.9754% 
가장 이론적인 손익

고점까지 도달 기간을 활용한 계산값 : 457min or 456.7527 + 1.96*434.7243 =  1308.8 = 1309min
너무 편차가 커서 사실상 평균 평균 이상부터는 손익이 기하급수적으로 늘 수도 있지만 잃을 확률이 그 이상으로 높아집니다

고점도달까지 등락폭을 활용한 계산 : -3.0543% + 1.96*2.2128% = 7.391
가장 현실적이나 평균적으로 0.611%의 수익을 얻자고 7.391%를 걸면 손익비가 12배로 너무 비효율적임. 하지만 비율을 잘 조정해서 저점해서 사는 방법도 고려해 볼 만 하다고 생각합니다.

결론 : 수익을 대폭 올릴 수 있는 가능성을 가지고, 쉽게 구현이 가능하지만 현실에서 적용한다면 이론적으로 필요한 조건들이 많다고 생각합니다. ex : 손절기준, 매도타이밍...
하지만 변동성 돌파로 확실하게 고점을 쉽게 챙긴다는 면에서 앞으로 결합에 좋은 포텐셜을 가질 수 있다고 생각합니다. 그리고 현실적으로 수수료 보다 높을 확률이 64%를 넘어서 수익을 얻을 수 있는 확률이 50%를 넘고, 이론상 기하급수적인 성장. 그리고 고점 돌파시 종가매매도 생각보다 단순한 매수매도이지만 거래량을 전체 0.01%로 제한했음에도 큰 수익률이 가능성을 증명한다고 생각합니다.